In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import torch
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import json

In [ ]:
suhi_combined_file_path = '/content/drive/MyDrive/SUHI/Data/CAP_Postdischarge_data_16FEB24.xlsx - coded.csv'
suhi_df = pd.read_csv(suhi_combined_file_path)

<ipython-input-2-b85ddacd002c>:2: DtypeWarning: Columns (100,110,130,147,161,168,173,178,194,198,214,226,254,259,274,276,283,293,297,303,310,366,379,381,384,386,390,392,401,403,406,411,414,417,419,422,424,427,430,433,435,438,444,446,449,451,454,456,459,461,468,472,474,476,480,484,486,489) have mixed types. Specify dtype option on import or set low_memory=False.
  suhi_df = pd.read_csv(suhi_combined_file_path)


In [ ]:
excluded_words = ['lvm', 'closed case', 'case closed', 'left a vm', 'no response', 'declined', 'no answer', 'pt unavailable']

suhi_df['combined_notes'] = suhi_df[suhi_df.columns[suhi_df.columns.str.contains('notes_contact')]].apply(
    lambda x: '. '.join([note for note in x.dropna().astype(str).str.lower()
                         if len(note.split()) >= 2 and not any(excluded_word in note for excluded_word in excluded_words)]),
    axis=1
)

In [ ]:
# Drop based on null values in combined_notes
suhi_without_null_df = suhi_df.dropna(subset=['combined_notes', 'day_readmit'])

In [ ]:
def remove_repeated_chars(text):
    return re.sub(r'(.)\1{2,}', '', str(text))

suhi_without_null_df['combined_notes'] = suhi_without_null_df['combined_notes'].apply(remove_repeated_chars)

<ipython-input-5-c831da4439a4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suhi_without_null_df['combined_notes'] = suhi_without_null_df['combined_notes'].apply(remove_repeated_chars)


In [ ]:
suhi_without_null_df.reset_index(drop=True, inplace=True)

In [ ]:
# Replacing short forms in each record
for record in range(len(suhi_without_null_df['combined_notes'])):
  if 'sdoh' in suhi_without_null_df['combined_notes'][record]:
    suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' sdoh ', ' social determinants of health ')
  if 'pt' in suhi_without_null_df['combined_notes'][record]:
    suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' pt ', ' patient ')
  if "pt's" in suhi_without_null_df['combined_notes'][record]:
    suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(" pt's ", ' patients ')
  if 'pcp' in suhi_without_null_df['combined_notes'][record]:
    suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' pcp ', ' primary care provider ')
  if 'lvm' in suhi_without_null_df['combined_notes'][record]:
    suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' lvm ', ' left voice mail ')

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' pt ', ' patient ')
<ipython-input-7-d8c9829c5050>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suhi_without_null_df['combined_notes'][record] = suhi_without_null_df['combined_notes'][record].replace(' pt ', ' patient ')
<ipython-input-7-d8c9829c5050>:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this 

In [ ]:
suhi_without_null_df

,7,redcap_event_name,new_patient,type_id,chw_name,inperson_connection,sw_date,referral_date,referral_month,referral_source,...,sdoh_other,sdoh_other_notes,resources_other,resources_other_notes,no_resources_other,no_resources_other_other,sdoh_notes,nowpow,old_sdoh_questionnaire_complete,combined_notes
0,2,round_1_arm_1,0.0,1.0,3.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,
1,4,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2,5,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,emotional support
3,5,round_2_arm_1,0.0,1.0,2.0,NaN,9/21/2021,9/23/2021,2109.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,
4,6,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2276,round_1_arm_1,1.0,1.0,4.0,0.0,1/19/2024,1/25/2024,2401.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,vm is not set up.. vm is not set up.
2524,2277,round_1_arm_1,1.0,1.0,4.0,0.0,1/19/2024,1/25/2024,2401.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,phone rang for a while and went to a busy sign...
2525,2279,round_1_arm_1,1.0,1.0,2.0,0.0,1/19/2024,1/25/2024,2401.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,
2526,2290,round_1_arm_1,1.0,1.0,4.0,0.0,1/26/2024,1/30/2024,2401.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,


In [ ]:
suhi_without_null_df['day_readmit'].value_counts()

,count
day_readmit,
0.0,1895
1.0,629
2.0,4


In [ ]:
# If day_readmit is 2 change it to 0
suhi_without_null_df.loc[suhi_without_null_df['day_readmit'] == 2, 'day_readmit'] = 0

In [ ]:
suhi_without_null_df['day_readmit'].value_counts()

,count
day_readmit,
0.0,1899
1.0,629


In [ ]:
suhi_without_null_df['non_null_empty'] = suhi_without_null_df['combined_notes'].apply(lambda x: x == '' or (isinstance(x, str) and x.isspace()))

<ipython-input-21-d97d739e5189>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suhi_without_null_df['non_null_empty'] = suhi_without_null_df['combined_notes'].apply(lambda x: x == '' or (isinstance(x, str) and x.isspace()))


In [ ]:
suhi_filtered_df = suhi_without_null_df[suhi_without_null_df['non_null_empty'] == False]

In [ ]:
suhi_filtered_df = suhi_filtered_df.drop('non_null_empty', axis=1)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3.5-mini-instruct",
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    num_workers=4
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "do_sample": False,
}

extracted_list = []
for record_num in tqdm(range(len(suhi_filtered_df[600:]))):

    prompt = f"""You are an AI assistant specialized in analyzing patient notes to identify specific health and social factors. Your task is to carefully read the provided patient note and determine the presence or absence of key factors. Respond with precision and objectivity.

                  Patient Note:
                  {suhi_filtered_df.iloc[record_num + 600]['combined_notes']}

                  Instructions:
                  1. Analyze the note thoroughly for explicit mentions or clear implications of each factor.
                  2. Use '1' to indicate presence and '0' to indicate absence of a factor.
                  3. Provide your response as a JSON object for easy parsing and analysis.
                  4. Include only the JSON object in your response, without any additional text or explanation.

                  Factors to identify:
                  "Food insecurity",
                  "Transportation issues",
                  "Unmet needs",
                  "Emotional needs (depression, anxiety, or loneliness)",
                  "Primary Care Physician (PCP) mentioned",
                  "Unemployment",
                  "Comorbidities",
                  "Hypertension",
                  "Diabetes",
                  "Asthma"

                  Important considerations:
                  - Be thorough in your analysis, considering both explicit statements and strong implications.
                  - Maintain consistency in your interpretations across different patient notes.
                  - If the information is ambiguous or unclear, default to '0' to avoid false positives.
                  - Your response should consist solely of the completed JSON object."""
    messages = [
        {"role": "system", "content": "You are a helpful AI content extractor to extract content from the provided context in medical domain. If there is no content revleant to the topic or statement given in the provided context return an empty string"},
        {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
        {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
        {"role": "user", "content": prompt},
    ]

    output = pipe(messages, **generation_args)
    print(output[0]['generated_text'])
    try:
        # Find the bounds of the JSON part
        prediction = output[0]['generated_text']
        json_start = prediction.index('{')
        json_end = prediction.rindex('}') + 1
        json_part = prediction[json_start:json_end]

        # Parse the JSON part to a dictionary
        data_dict = json.loads(json_part)
        print(50*'-')
        print(record_num)
        print(suhi_filtered_df.iloc[record_num]['combined_notes'])
        print(data_dict)
        extracted_list.append(data_dict)

    except ValueError as e:
        print("Error processing the input string:", e)
        extracted_list.append('')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 1/1168 [00:49<15:54:46, 49.09s/it]

 ```json

{

  "Food insecurity": 0,

  "Transportation issues": 0,

  "Unmet needs": 0,

  "Emotional needs (depression, anxiety, or loneliness)": 1,

  "Primary Care Physician (PCP) mentioned": 0,

  "Unemployment": 0,

  "Comorbidities": 0,

  "Hypertension": 0,

  "Diabetes": 0,

  "Asthma": 0

}

```
--------------------------------------------------
0
emotional support
{'Food insecurity': 0, 'Transportation issues': 0, 'Unmet needs': 0, 'Emotional needs (depression, anxiety, or loneliness)': 1, 'Primary Care Physician (PCP) mentioned': 0, 'Unemployment': 0, 'Comorbidities': 0, 'Hypertension': 0, 'Diabetes': 0, 'Asthma': 0}


  0%|          | 2/1168 [01:37<15:42:00, 48.47s/it]

 ```json

{

  "Food insecurity": 0,

  "Transportation issues": 1,

  "Unmet needs": 1,

  "Emotional needs (depression, anxiety, or loneliness)": 0,

  "Primary Care Physician (PCP) mentioned": 0,

  "Unemployment": 0,

  "Comorbidities": 0,

  "Hypertension": 0,

  "Diabetes": 0,

  "Asthma": 0

}

```
--------------------------------------------------
1
looking for housing in the same and needs access to an elevator for her powered wheelchair.. the patient reached out to me to let me know she is currently back in the hospital. i reassured her that i'm looking for housing for her. i spoke with the patient about where the status of her  cha application.
{'Food insecurity': 0, 'Transportation issues': 1, 'Unmet needs': 1, 'Emotional needs (depression, anxiety, or loneliness)': 0, 'Primary Care Physician (PCP) mentioned': 0, 'Unemployment': 0, 'Comorbidities': 0, 'Hypertension': 0, 'Diabetes': 0, 'Asthma': 0}


  0%|          | 3/1168 [02:25<15:36:27, 48.23s/it]

 ```json

{

  "Food insecurity": 0,

  "Transportation issues": 0,

  "Unmet needs": 1,

  "Emotional needs (depression, anxiety, or loneliness)": 1,

  "Primary Care Physician (PCP) mentioned": 0,

  "Unemployment": 0,

  "Comorbidities": 0,

  "Hypertension": 0,

  "Diabetes": 0,

  "Asthma": 0

}

```
--------------------------------------------------
2
sdoh/ emotional support. emotiional support/ case cloud
{'Food insecurity': 0, 'Transportation issues': 0, 'Unmet needs': 1, 'Emotional needs (depression, anxiety, or loneliness)': 1, 'Primary Care Physician (PCP) mentioned': 0, 'Unemployment': 0, 'Comorbidities': 0, 'Hypertension': 0, 'Diabetes': 0, 'Asthma': 0}


  0%|          | 3/1168 [02:47<18:04:10, 55.84s/it]


KeyboardInterrupt: 

In [ ]:
generative_topic_df = pd.DataFrame(extracted_list)

In [ ]:
generative_topic_df

,Food insecurity,Transportation issues,Unmet needs,"Emotional needs (depression, anxiety, or loneliness)",Primary Care Physician (PCP) mentioned,Unemployment,Comorbidities,Hypertension,Diabetes,Asthma
0,0,0,0,1,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0
2,0,0,1,1,0,0,0,0,0,0


In [ ]:
suhi_filtered_df[600:]

,7,redcap_event_name,new_patient,type_id,chw_name,inperson_connection,sw_date,referral_date,referral_month,referral_source,...,sdoh_other,sdoh_other_notes,resources_other,resources_other_notes,no_resources_other,no_resources_other_other,sdoh_notes,nowpow,old_sdoh_questionnaire_complete,combined_notes
2,5,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,emotional support
6,9,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,looking for housing in the same and needs acce...
7,10,round_1_arm_1,1.0,1.0,2.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,sdoh/ emotional support. emotiional support/ c...
9,12,round_1_arm_1,1.0,1.0,4.0,NaN,4/12/2021,4/12/2021,2104.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,"Patients PCP is out of town, patient is waitin...",0.0,2,check to see if primary care provider has retu...
10,12,round_2_arm_1,1.0,1.0,5.0,NaN,5/7/2021,5/14/2021,2105.0,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2,pt stated she is ok- doesn't need any assistan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,1551,round_1_arm_1,1.0,1.0,2.0,0.0,3/9/2022,3/10/2022,2203.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,"the patient wasn't feeling well, she communica..."
1769,1551,round_2_arm_1,1.0,1.0,2.0,0.0,4/11/2023,4/12/2023,2304.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,i was able to get connected to the pts son mr....
1770,1552,round_1_arm_1,1.0,1.0,3.0,0.0,3/9/2022,3/10/2022,2203.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,pt's contact is a wrong #. left message with p...
1771,1552,round_2_arm_1,0.0,1.0,10.0,0.0,8/10/2022,8/10/2022,2208.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,i called patient at number in nextgen 773.290....


In [ ]:
generated_topics_df2 = pd.concat([generative_topic_df, suhi_filtered_df[600:]], axis=1)

In [ ]:
generated_topics_df2

,Food insecurity,Transportation issues,Unmet needs,"Emotional needs (depression, anxiety, or loneliness)",Primary Care Physician (PCP) mentioned,Unemployment,Comorbidities,Hypertension,Diabetes,Asthma,...,sdoh_other,sdoh_other_notes,resources_other,resources_other_notes,no_resources_other,no_resources_other_other,sdoh_notes,nowpow,old_sdoh_questionnaire_complete,combined_notes
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,emotional support
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,looking for housing in the same and needs acce...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.0,sdoh/ emotional support. emotiional support/ c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,"the patient wasn't feeling well, she communica..."
1769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,i was able to get connected to the pts son mr....
1770,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,pt's contact is a wrong #. left message with p...
1771,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,i called patient at number in nextgen 773.290....


In [ ]:
import pandas as pd

In [ ]:
generated_topics_reasoning_df = pd.read_csv('/content/CAP_Postdischarge_data_16FEB24_generated_topics_reasoning_df1.csv')

<ipython-input-3-f7528bbab65b>:1: DtypeWarning: Columns (16,121,131,141,151,182,189,194,199,211,215,219,235,247,261,275,278,295,297,304,314,318,324,331,387,395,400,402,405,407,411,413,422,424,427,432,435,438,440,443,445,448,451,454,456,459,465,470,472,475,477,480,482,489,493,495,497,501,505,507,510) have mixed types. Specify dtype option on import or set low_memory=False.
  generated_topics_reasoning_df = pd.read_csv('/content/CAP_Postdischarge_data_16FEB24_generated_topics_reasoning_df1.csv')


In [ ]:
generated_topics_sentiment_df = pd.read_csv('/content/CAP_Postdischarge_data_16FEB24_generated_topics_sentiment_df1.csv')

<ipython-input-4-6f94c09c3a45>:1: DtypeWarning: Columns (16,121,131,141,151,182,189,194,199,211,215,219,235,247,261,275,278,295,297,304,314,318,324,331,387,395,400,402,405,407,411,413,422,424,427,432,435,438,440,443,445,448,451,454,456,459,465,470,472,475,477,480,482,489,493,495,497,501,505,507,510) have mixed types. Specify dtype option on import or set low_memory=False.
  generated_topics_sentiment_df = pd.read_csv('/content/CAP_Postdischarge_data_16FEB24_generated_topics_sentiment_df1.csv')


## Value Counts

In [ ]:
for col in generated_topics_reasoning_df.columns[:15]:
  print(50*'-')
  print(generated_topics_reasoning_df[col].value_counts())

--------------------------------------------------
combined_notes
will call again.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
for col in generated_topics_sentiment_df.columns[:15]:
  print(50*'-')
  print(generated_topics_sentiment_df[col].value_counts())

--------------------------------------------------
combined_notes
will call again.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

## Value counts by readmission without Sentiment

In [ ]:
# Columns for which you want value counts
columns_of_interest = generated_topics_reasoning_df.columns[1:15]

# Get value counts for each column grouped by 'target'
value_counts_by_target = {col: generated_topics_reasoning_df.groupby('day_readmit')[col].value_counts().unstack(fill_value=0) for col in columns_of_interest}

# Display value counts for each column
for col, counts in value_counts_by_target.items():
    print(f"Value counts for {col} grouped by 'day_readmit':\n{counts}\n")

Value counts for Food insecurity grouped by 'day_readmit':
Food insecurity    0   1
day_readmit             
0                885  81
1                184  18

Value counts for Transportation grouped by 'day_readmit':
Transportation    0    1
day_readmit             
0               669  297
1               140   62

Value counts for Housing grouped by 'day_readmit':
Housing        0    1
day_readmit          
0            770  196
1            157   45

Value counts for Emotional needs (depression, anxiety, or loneliness) grouped by 'day_readmit':
Emotional needs (depression, anxiety, or loneliness)  0.0  1.0
day_readmit                                                   
0                                                     793  172
1                                                     144   58

Value counts for Depression grouped by 'day_readmit':
Depression     0   1
day_readmit         
0            928  38
1            190  12

Value counts for Anxiety grouped by 'day_readmit':
An

## Value counts by readmission with Sentiment

In [ ]:
# Columns for which you want value counts
columns_of_interest = generated_topics_sentiment_df.columns[1:15]

# Get value counts for each column grouped by 'target'
value_counts_by_target = {col: generated_topics_sentiment_df.groupby('day_readmit')[col].value_counts().unstack(fill_value=0) for col in columns_of_interest}

# Display value counts for each column
for col, counts in value_counts_by_target.items():
    print(f"Value counts for {col} grouped by 'day_readmit':\n{counts}\n")

Value counts for Food insecurity grouped by 'day_readmit':
Food insecurity    0   1   2   3
day_readmit                     
0                866  63  14  23
1                179   9   7   7

Value counts for Transportation grouped by 'day_readmit':
Transportation    0    1   2   3
day_readmit                     
0               683  144  85  54
1               142   25  22  13

Value counts for Housing grouped by 'day_readmit':
Housing        0   1   2    3
day_readmit                  
0            688  86  39  153
1            141  13  18   30

Value counts for Emotional needs (depression, anxiety, or loneliness) grouped by 'day_readmit':
Emotional needs (depression, anxiety, or loneliness)  0.0  1.0  2.0  3.0
day_readmit                                                             
0                                                     536   25   63  339
1                                                      95    4   21   82

Value counts for Depression grouped by 'day_readmit':
De